# Соревнования Kaggle Сентимент-анализ отзывов на товары (простая версия)

Для начала загрузим данные и посмотрим на них

In [1]:
import pandas as pd

Так будет выглядеть наше решение :)

In [2]:
data = pd.read_csv(r'D:\Pyton\Learning\Course_6\Week_3\products_sentiment_sample_submission.csv')

In [3]:
data.head(5)

,Id,y
0,0,0
1,1,1
2,2,0
3,3,1
4,4,0


In [27]:
data.shape

(500, 2)

In [3]:
train = pd.read_csv(r'D:\Pyton\Learning\Course_6\Week_3\products_sentiment_train.tsv', delimiter='\t', names=["text", "label"])

In [4]:
train.head(5)

,text,label
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [19]:
train['label'][:10]

0    1
1    1
2    1
3    0
4    1
5    1
6    1
7    1
8    1
9    1
Name: label, dtype: int64

In [29]:
train.shape

(2000, 2)

In [5]:
test = pd.read_csv(r'D:\Pyton\Learning\Course_6\Week_3\products_sentiment_test.tsv', delimiter='\t')

In [6]:
test.head(5)

,Id,text
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


In [26]:
test['text'][1]

'3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze . '

In [30]:
test.tail(10)

,Id,text
294,294,the service from t-mobile is very good too .
295,295,yes there is a corner lens barrel distortion (...
296,296,the zx boasts superior sound quality than the ...
297,297,definetely a great camera .
298,298,"we hardly ever have to charge them , get great..."
299,299,the diaper champ is the best !
300,300,the only problem i had was a small glitch with...
301,301,8mb for a camera like this is a joke .
302,302,i like the size and weight of this little crit...
303,303,7- 360


# 2. Начинаем работать с тренировочными данными

Для дальнейшей работы подготовим 2 списка. Первый из них список текстов в порядке их следования в датасете:

In [7]:
t = train['text'].tolist()
t[:10]

['2 . take around 10,000 640x480 pictures .',
 'i downloaded a trial version of computer associates ez firewall and antivirus and fell in love with a computer security system all over again .',
 'the wrt54g plus the hga7t is a perfect solution if you need wireless coverage in a wider area or for a hard-walled house as was my case .',
 'i dont especially like how music files are unstructured ; basically they are just dumped into one folder with no organization , like you might have in windows explorer folders and subfolders .',
 'i was using the cheapie pail ... and it worked ok until the opening device fell apart .',
 'you can manage your profile , change the contrast of backlight , make different type of display , either list or tabbed . ',
 '11 ) minimal pause between songs - less than that on other devices - good achievement for a hard-drive based player ',
 'from the internet , woodworking books , local stores and personal opinions the hitachi m12v 3-1 / 4 hp router time and time a

Второй список меток классов соответствующих сообщениям из списка 1

In [8]:
label = train['label'].tolist()
label[:10]

[1L, 1L, 1L, 0L, 1L, 1L, 1L, 1L, 1L, 1L]

**Модель "мешка слов"**  
Мы импортируем класс CountVectorizer, создам экземпляр класса и подгоняем модель к нашим данным. Т.е. используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получаем из списка текстов матрицу признаков X.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [37]:
vect = CountVectorizer().fit(t)
X_train = vect.transform(t)

Применим **LogisticRegression** с использование перекресной проверки 

In [40]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC

In [59]:
accuracy = cross_val_score(LogisticRegression(random_state = 2), X_train, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))
#print("Средняя точность перекр проверки: {:.2f}".format(np.mean(scores)))

Средняя точность перекр проверки: 0.77


In [39]:
accuracy = cross_val_score(LinearSVC(), X_train, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя правильность перекр проверки: 0.75


In [42]:
accuracy = cross_val_score(SGDClassifier(), X_train, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy)))

C:\Users\Vera\Anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Средняя правильность перекр проверки: 0.73


Средняя точность 0.77, результат неплохой.

Логистическая регрессия имеет параметра регуляризации С, который можно настроить с помощью перекресной проверки

In [19]:
from sklearn.model_selection import GridSearchCV 
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]} 
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5) 
grid.fit(X_train,label) 
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры:{:.2f} ", grid.best_params_)

Наилучшее значение перекрестной проверки: 0.77
('\xd0\x9d\xd0\xb0\xd0\xb8\xd0\xbb\xd1\x83\xd1\x87\xd1\x88\xd0\xb8\xd0\xb5 \xd0\xbf\xd0\xb0\xd1\x80\xd0\xb0\xd0\xbc\xd0\xb5\xd1\x82\xd1\x80\xd1\x8b:{:.2f} ', {'C': 1})


Используем только те токены, которые встечаются как минимум в 5, 10 документах

In [24]:
vect = CountVectorizer(min_df=5).fit(t) 
X_train = vect.transform(t) 

In [21]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5) 
grid.fit(X_train, label)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))

Наилучшее значение перекрестной проверки: 0.76


In [25]:
accuracy = cross_val_score(LogisticRegression(), X_train, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя точность перекр проверки: 0.76


In [22]:
vect = CountVectorizer(min_df=10).fit(t) 
X_train = vect.transform(t) 
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5) 
grid.fit(X_train, label)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))

Наилучшее значение перекрестной проверки: 0.76


In [23]:
vect = CountVectorizer(min_df=50).fit(t) 
X_train = vect.transform(t) 
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5) 
grid.fit(X_train, label)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))

Наилучшее значение перекрестной проверки: 0.72


**Стоп слова**

In [26]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [27]:
vect = CountVectorizer(min_df=5, stop_words="english").fit(t) 
X_train = vect.transform(t)

In [28]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, label) 
print("Наилучшее значение перекр проверки: {:.2f}".format(grid.best_score_)) 

Наилучшее значение перекр проверки: 0.74


In [29]:
vect = CountVectorizer(stop_words="english").fit(t) 
X_train = vect.transform(t)
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, label) 
print("Наилучшее значение перекр проверки: {:.2f}".format(grid.best_score_)) 

Наилучшее значение перекр проверки: 0.75


In [30]:
accuracy = cross_val_score(LogisticRegression(), X_train, label, cv = 5)
print("Средняя точность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя точность перекр проверки: 0.75


In [51]:
from nltk.corpus import stopwords


In [52]:
stopwords = stopwords.words('english')
print(stopwords[:10])

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u"you're"]


In [54]:
vect_nltk_sw = CountVectorizer(stop_words = stopwords)
X_train_nltk_sw = vect_nltk_sw.fit_transform(t)
accuracy_nltk_sw = cross_val_score(LogisticRegression(), X_train_nltk_sw, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy_nltk_sw)))

Средняя правильность перекр проверки: 0.75


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
from sklearn.pipeline import make_pipeline 
pipe = make_pipeline(TfidfVectorizer(norm=None),LogisticRegression()) 
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]} 
grid = GridSearchCV(pipe, param_grid, cv=5) 
grid.fit(t, label) 
print("Наилучшее значение перекр проверки: {:.2f}".format(grid.best_score_)) 

Наилучшее значение перекр проверки: 0.77


n-граммы

In [35]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
# выполнение решетчатого поиска займет много времени из - за # относительно большой сетки параметров и включения триграмм 
param_grid = {"logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100], "tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)]} 
 
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(t, label) 
print("Наилучшее значение перекр проверки: {:.2f}".format(grid.best_score_)) 
print("Наилучшие параметры:\n{}".format(grid.best_params_)) 

Наилучшее значение перекр проверки: 0.77
Наилучшие параметры:
{'tfidfvectorizer__ngram_range': (1, 1), 'logisticregression__C': 1}


In [48]:
cv = CountVectorizer(ngram_range=(1,3))
X_train_bi = cv.fit_transform(t)
accuracy_bi = cross_val_score(LogisticRegression(), X_train_bi, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy_bi)))

Средняя правильность перекр проверки: 0.77


In [47]:
accuracy = cross_val_score(LinearSVC(), X_train_bi, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя правильность перекр проверки: 0.76


In [49]:
accuracy = cross_val_score(SGDClassifier(), X_train, label, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(accuracy)))

Средняя правильность перекр проверки: 0.75


Тестовые данные

In [55]:
test_data = test['text'].tolist()
test_data[:10]

["so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around .",
 '3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze . ',
 'better for the zen micro is outlook compatibility .',
 '6 . play gameboy color games on it with goboy .',
 "likewise , i 've heard norton 2004 professional version is fine too .",
 'mine was 2 weeks old and i chucked it in the trash , where it belongs . ',
 "i find it very stable and comfortable to use , it doesn 't jump around or make me feel like i need to use a lot of arm strength to keep it under control , and it has very good visibility to see your cut .",
 "styling / ergonomics : the keys are small , which generally doesn 't bother me if they are spaced because i can employ a little ' braille , ' but the keys run into each other , so careful attn. must be paid when dialing ."

In [56]:
vect = CountVectorizer().fit(test_data)
X_test = vect.transform(test_data)

In [60]:
clf = LogisticRegression(random_state = 2)
clf.fit(X_train, label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
clf.predict(X_test)

ValueError: X has 1971 features per sample; expecting 3973